In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
y_train = np.ravel(pd.read_csv('../data/y_train.csv', index_col=0))
X_test = pd.read_csv('../data/X_test.csv', index_col=0)
y_test = np.ravel(pd.read_csv('../data/y_test.csv', index_col=0))

In [3]:
lr = LogisticRegression(random_state=17)
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      3635
           1       0.66      0.53      0.59      1295

    accuracy                           0.81      4930
   macro avg       0.75      0.72      0.73      4930
weighted avg       0.80      0.81      0.80      4930



In [4]:
y_test_pred = lr.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1539
           1       0.65      0.48      0.56       574

    accuracy                           0.79      2113
   macro avg       0.74      0.69      0.71      2113
weighted avg       0.78      0.79      0.78      2113



In [5]:
rfc = RandomForestClassifier(random_state=17)
rfc.fit(X_train, y_train)
y_train_pred = rfc.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3635
           1       0.99      0.99      0.99      1295

    accuracy                           1.00      4930
   macro avg       1.00      0.99      0.99      4930
weighted avg       1.00      1.00      1.00      4930



In [6]:
y_test_pred = rfc.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1539
           1       0.61      0.46      0.53       574

    accuracy                           0.77      2113
   macro avg       0.71      0.68      0.69      2113
weighted avg       0.76      0.77      0.76      2113

